In [19]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom/original/yolox_voc_tiny.py',
ckpt_path = '/workspace/tensorrt/YOLOX/best_ckpt.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

In [20]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [21]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [22]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [23]:
from pruning_imagenet import Mask

In [24]:
m = Mask(model)

In [25]:
m.init_length()

In [26]:
m.init_mask(1, 0.21)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done


In [27]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [28]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 24, number of nonzero weight is 3648, zero is 960
layer: 27, number of nonzero weight is 3648, zero is 960
layer: 30, number of nonzero weight is 7296, zero is 1920
layer: 33, number of nonzero weight is 1824, zero is 480
layer: 39, number of nonzero weight is 1824, zero is 480
layer: 45, number of nonzero weight is 1824, zero is 480
layer: 54, number of nonzero weight is 14592, zero is 3840
layer: 57, number of nonzero weight is 14592, zero is 3840
layer: 60, number of nonzero weight is 29184, zero is 7680
layer: 63, number of nonzero weight is 7296, zero is 1920
layer: 69, number of nonzero weight is 7296, zero is 1920
layer: 75, number of nonzero weight is 7296, zero is 1920
layer: 90, number of nonzero weight is 58368, zero is 15360
layer: 93, number of nonzero weight is 58368, zero is 15360
layer: 96, number of nonzero weight is 116736, zero is 30720
layer: 99, number of nonzero weight is 29184, zero is 7680
layer: 102, number of nonzero weight is 262656, zero is 69120
laye

In [29]:
model = m.model


In [30]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            temp
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp

pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channex index :  0
pruning channex index :  22
pruning channex index :  23
pruning channex index :  24
pruning channex index :  25
pruning channex index :  26
pruning channex index :  30
pruning channex index :  31
pruning channex index :  37
pruning ch

In [31]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv1.conv.weight': [0, 22, 23, 24, 25, 26, 30, 31, 37, 42], 'backbone.backbone.dark3.1.conv2.conv.weight': [6, 20, 26, 29, 30, 31, 34, 37, 38, 41], 'backbone.backbone.dark3.1.conv3.conv.weight': [1, 8, 12, 15, 17, 18, 21, 29, 34, 37, 51, 55, 65, 66, 67, 70, 79, 81, 90, 95], 'backbone.backbone.dark3.1.m.0.conv1.conv.weight': [4, 14, 18, 21, 27, 36, 37, 38, 41, 43], 'backbone.backbone.dark3.1.m.1.conv1.conv.weight': [1, 11, 12, 16, 18, 23, 26, 33, 38, 42], 'backbone.backbone.dark3.1.m.2.conv1.conv.weight': [2, 12, 15, 27, 29, 30, 33, 34, 36, 40], 'backbone.backbone.dark4.1.conv1.conv.weight': [4, 9, 17, 19, 23, 24, 25, 26, 31, 46, 59, 63, 64, 66, 72, 75, 80, 84, 85, 89], 'backbone.backbone.dark4.1.conv2.conv.weight': [1, 2, 9, 14, 19, 26, 35, 39, 42, 43, 48, 56, 72, 74, 75, 84, 85, 86, 92, 94], 'backbone.backbone.dark4.1.conv3.conv.weight': [4, 5, 10, 19, 20, 21, 34, 39, 41, 45, 63, 64, 72, 83, 84, 89, 95, 99, 101, 102, 103, 106, 113, 118, 120, 122, 132, 148,

In [32]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)

['backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[1].conv1.conv', 'backbone.backbone.dark4[1].m[2].conv1.conv', 'backbone.backbone.dark5[2].conv1.conv', 'backbone.backbone.dark5[2].conv2.conv', 'backbone.backbone.dark5[2].conv3.conv', 'backbone.backbone.dark5[2].m[0].conv1.conv', 'backbone.backbone.dark5[2].m[0].conv2.conv', 'backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv', 'backbone.C3_p4.conv2.conv', 'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m[0].conv1.conv', 'backbone.C3_p4.m[0].conv2.conv', 'backbone.reduce_conv1.conv', 'backbone.bu_conv2.conv', 'backbone.bu_conv1.c

In [33]:
value = list(dict.values())
value

[[0, 22, 23, 24, 25, 26, 30, 31, 37, 42],
 [6, 20, 26, 29, 30, 31, 34, 37, 38, 41],
 [1,
  8,
  12,
  15,
  17,
  18,
  21,
  29,
  34,
  37,
  51,
  55,
  65,
  66,
  67,
  70,
  79,
  81,
  90,
  95],
 [4, 14, 18, 21, 27, 36, 37, 38, 41, 43],
 [1, 11, 12, 16, 18, 23, 26, 33, 38, 42],
 [2, 12, 15, 27, 29, 30, 33, 34, 36, 40],
 [4,
  9,
  17,
  19,
  23,
  24,
  25,
  26,
  31,
  46,
  59,
  63,
  64,
  66,
  72,
  75,
  80,
  84,
  85,
  89],
 [1, 2, 9, 14, 19, 26, 35, 39, 42, 43, 48, 56, 72, 74, 75, 84, 85, 86, 92, 94],
 [4,
  5,
  10,
  19,
  20,
  21,
  34,
  39,
  41,
  45,
  63,
  64,
  72,
  83,
  84,
  89,
  95,
  99,
  101,
  102,
  103,
  106,
  113,
  118,
  120,
  122,
  132,
  148,
  149,
  151,
  152,
  162,
  168,
  170,
  172,
  177,
  181,
  183,
  186,
  188],
 [0,
  20,
  22,
  37,
  39,
  40,
  41,
  47,
  49,
  53,
  54,
  60,
  62,
  63,
  71,
  72,
  73,
  75,
  91,
  93],
 [18,
  27,
  29,
  33,
  40,
  41,
  43,
  44,
  47,
  48,
  50,
  54,
  56,
  58,
  60,
 

In [34]:
for i in range(len(temp)):
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, value[i])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     


Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[0, 22, 23, 24, 25, 26, 30, 31, 37, 42], metric={'#params': 960}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[0, 22, 23, 24, 25, 26, 30, 31, 37, 42], metric={'#params': 20}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats

In [35]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [36]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, '0.2.pth')


